In [ ]:
!pip install --upgrade nibabel

Get data


In [ ]:
!wget "https://putpoznanpl-my.sharepoint.com/:u:/g/personal/dominik_pieczynski_put_poznan_pl/EWIZ_xm8wXpMjQDgF2VQ1csB4QuHPKoj5vDpj6CQi9p-AA?e=yQr6fn&download=1" -O public.zip
!unzip -q public.zip
!rm public.zip

Create valid dirs

In [ ]:
!mkdir /content/FirstDataset/valid
!mkdir /content/SecondDataset/valid

Checking the number of scans in dirs 

In [ ]:
import os
import shutil

print("FirstDataset")
filepath1 = '/content/FirstDataset/train'
filenames1 = os.listdir(filepath1)
filenames1 = sorted(filenames1)
print('train', len(filenames1))
print('test', len(os.listdir('/content/FirstDataset/test')))
print(int(len(filenames1)/5)) #114

print("SecondDataset")
filepath2 = '/content/SecondDataset/train'
filenames2 = os.listdir(filepath2)
filenames2 = sorted(filenames2)
print('train', len(filenames2))
print('test', len(os.listdir('/content/SecondDataset/test')))
print(int(len(filenames2)/5)) # 20

Transfer of some data from train dir to valid dir

In [ ]:
import os
import shutil

# FirstDataset
filepath1 = '/content/FirstDataset/train'
filenames1 = os.listdir(filepath1)
filenames1 = sorted(filenames1)
# print(len(filenames1))
valid_filenames1 = filenames1[:int(len(filenames1)/5)]
# print(valid_filenames1)
for file_name in valid_filenames1:
  shutil.move(f"/content/FirstDataset/train/{file_name}", f"/content/FirstDataset/valid/{file_name}")
print("train len:", len(os.listdir('/content/FirstDataset/train')))
print("valid len:", len(os.listdir('/content/FirstDataset/valid')))
print('test len', len(os.listdir('/content/FirstDataset/test')))
print('finish 1')

# SecondDataset
filepath2 = '/content/SecondDataset/train'
filenames2 = os.listdir(filepath2)
filenames2 = sorted(filenames2)
# print(len(filenames2))
valid_filenames2 = filenames2[:int(len(filenames2)/5)]
# print(valid_filenames2)
for file_name in valid_filenames2:
  shutil.move(f"/content/SecondDataset/train/{file_name}", f"/content/SecondDataset/valid/{file_name}")
print("train len:", len(os.listdir('/content/SecondDataset/train')))
print("valid len:", len(os.listdir('/content/SecondDataset/valid')))
print('test len', len(os.listdir('/content/SecondDataset/test')))
print('finish 2')